# 20181010 Cellular Consortia Growth

### Goal
* define how easy / hard it will be to maintain multiple cells in the same culture given that they're growing at different rates

### Experimental Conditions
* exponential growth in liquid culture

### Approach
* Do simple dynamical simulations of cells growing at different rates in exponential phase
* analyze how much their population proportions vary in the culture

### Basic equation(s)

$$ x(t) = x_0 2^{t / T} $$

Where $T$ is the doubling time.

## Findings
* the fastest growing strain in a turbidostat will quickly overtake the rest of the population
* won't be able to just mix them all together

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

import numpy as np
import scipy as sp
import scipy.integrate

## Model Differential Growth Rates in Exponential
* model how different doubling times will diverge

In [ ]:
# Define exponential growth function

def exp_gr(time,dbl_time):
    return 

In [ ]:
# Create set of doubling times (in minutes)
dbl_ts = np.linspace(20,35,10)

# timespan of interest (in minutes)
t_span = np.linspace(0,300,100)

x_0 = 0.001

In [ ]:
# Create data structure to store growth data; Cols are dbling times; rows are time
growth_data = np.zeros((len(t_span), len(dbl_ts)))

# for loop to get the data
j = 0
for dbl_t in dbl_ts:
    i = 0
    for t in t_span:
        growth_data[i,j] = x_0 * 2**(t/dbl_t)
        i = i + 1
    j = j + 1

In [ ]:
# Plot them
fig, ax = plt.subplots()

i = 0
for dbl_t in dbl_ts:
    ax.plot(t_span, growth_data[:,i], label = dbl_t)
    i = i + 1
    
plt.xlabel('Time (min)')
plt.ylabel('OD')
plt.legend(title='Doubling time');

### Results
* They diverge quite a bit, but this doesn't exactly replicate what would occur in a turbidostat with dilution dependent on total OD

## Model Differential Growths in Turbidostat
* set a dilution rate such at that the SUM of all the ODs of the strains is constant
* initialize with different growth rates and initial concentrations
* set 4 different cell types

### Proportional Feedback control
* utilize proportional feedback control

For a given strain concentration, $x_i$, with growth rate $\phi_i$, the dynamics will be:

$$\dot{x_i} = \phi_i x_i - x_i u$$

Where $u$ is the feedback controller input with reference value $r$:

$$ u = k_p (x_{tot} - r)$$

Note that feedback control is actuated on the TOTAL concentration of all strains $x_{tot} = \sum x_i$

$\phi_i$ relation to doubling time $T_i$:

$$ \phi_i = ln(2) / T_i$$

### Single strain control
Figure out a good proportional feedback control gain by simulating a single strain.

#### System

$$\dot{x} = \phi x - x k_p(x-r)$$

Must set a constraint such that if desired value is below the reference, the input cannot ADD growth rate. Piecewise: If $x < r$, then $u = 0$, otherwise: $ u = k_p(x-r)$

In [ ]:
def ode_sys(t,x,params):
    # Calculate input (make sure it can't be positive)
    err = x[0] - params['r']
    if err > 0:
        u = params['k_p']*err
    else:
        u = 0
    
    x_dot = params['phi']*x[0] - x[0]*(u)
    return x_dot

#### Simulation

In [ ]:
# Set system parameters
dbl_t = 30 # minutes
growth_r = np.log(2)/dbl_t
params = {'phi': growth_r,
          'k_p': 1,
          'r': 0.5}

# Set ODE parameters
t_span = (0,300) # minutes
x0 = [0.4]

x_sol = sp.integrate.solve_ivp(fun = lambda t,y: ode_sys(t,y,params),t_span = t_span,y0=x0, max_step = 1)

#### Plot

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_sol.t,x_sol.y[0]);
ax.set_xlabel('Time (min)')
ax.set_ylabel('OD');

#### Results
* theres a steady-state error (of course, its only P control)
* try to use integral control as well

## PI Control

Must add new dynamics term $z$ to store the error over time:

$$\dot{z} = x_{tot} - r = e$$

Now the new input term:

$$u = k_p e + k_i \int_{0}^t e d\tau = k_p e + k_i z $$

### Single strain

#### System

\begin{align}
\dot{x} &= \phi x - x (k_p(x-r) + k_i z)\\
\dot{z} &= x -r
\end{align}

In [ ]:
def ode_sys(t,x,params):
    # Calculate input (make sure it can't be positive)
    err = x[0] - params['r']
    u = params['k_p']*err + params['k_i']*x[1]
    if u < 0:
        u = 0
    
    x_dot = params['phi']*x[0] - x[0]*(u)
    z_dot = x[0] - params['r']
    
    return [x_dot,z_dot]

In [ ]:
# Set system parameters
dbl_t = 40 # minutes
growth_r = np.log(2)/dbl_t
params = {'phi': growth_r,
          'k_p': 0.5,
          'k_i': .1,
          'r': 0.3}

# Set ODE parameters
t_span = (0,500) # minutes
x0 = [0.3,0]

x_sol = sp.integrate.solve_ivp(fun = lambda t,y: ode_sys(t,y,params),t_span = t_span,y0=x0, max_step = 0.1)

#### Plot

In [ ]:
# Plot it
fig, axes = plt.subplots(4,1, sharex = True)

axes[0].plot(x_sol.t,x_sol.y[0]);
axes[0].set_ylabel('OD')

axes[1].plot(x_sol.t,params['k_p']*(x_sol.y[0] - params['r']));
axes[1].set_ylabel('P term')

axes[2].plot(x_sol.t,params['k_i']*(x_sol.y[1]))
axes[2].set_ylabel('I term')

axes[3].plot(x_sol.t,(params['k_p']*(x_sol.y[0] - params['r']) + params['k_i']*(x_sol.y[1])))
axes[3].set_ylabel('input')
axes[3].set_xlabel('Time (min)');

axes[0].axhline(params['r'], linestyle=':',color='k');

#### Results
* have a controller with parameters that get the OD value to the setpoint pretty well (a bit of integrator windup but that's hard to avoid)

### Multiple strains
* each strain (let there be 4) has its own growth rate
* actuatation of the system is based on the total OD of the system ($x_{tot}$)

#### System

\begin{align}
\dot{x_i} &= \phi_i x_i - x_i (k_p(x_{tot}-r) + k_i z)\\
\dot{z} &= x_{tot} -r
\end{align}

In [ ]:
def ode_sys(t,x,params):
    # Calculate input (make sure it can't be positive)
    ref, k_i, k_p, phi, z = params['r'], params['k_i'], params['k_p'], params['phi'], x[-1]

    err = sum(x[0:3]) - ref
    u = k_p*err + k_i*z
    if u < 0:
        u = 0
    
    x0_dot = phi[0]*x[0] - x[0]*(u)
    x1_dot = phi[1]*x[1] - x[1]*(u)
    x2_dot = phi[2]*x[2] - x[2]*(u)
    x3_dot = phi[3]*x[3] - x[3]*(u)
    z_dot = err
    
    return [x0_dot,x1_dot,x2_dot,x3_dot,z_dot]

In [ ]:
# Set system parameters
dbl_ts = [38,39,40,41] # minutes
growth_r = np.log(2)/dbl_ts
params = {'phi': growth_r,
          'k_p': .5,
          'k_i': .1,
          'r': 0.4} # reference value


# Set ODE parameters
t_span = (0,1000) # minutes
x0 = [0.12,0.12,0.12,0.12,0] # initial values

x_sol = sp.integrate.solve_ivp(fun = lambda t,y: ode_sys(t,y,params),t_span = t_span,y0=x0, max_step = 0.1)

In [ ]:
# Plot it
fig, ax = plt.subplots()
ax.plot(x_sol.t,x_sol.y[0],x_sol.t,x_sol.y[1],x_sol.t,x_sol.y[2],x_sol.t,x_sol.y[3]);
ax.set_xlabel('Time (min)')
ax.set_ylabel('OD')
ax.axhline(params['r']/4, linestyle=':',color='k', label="reference")
ax.legend(dbl_ts + ['reference'], title="Doubling time");

#### Results
* even with doubling times only a minute apart, the fastest growing cells quickly take over
* this won't work if all cells are just mixed in at the beginning
* also for some reason I'm not defining the reference correctly, but that doesn't really matter

## Bounding OD Values
* instead of using PI controllers to control, try setting upper and lower OD values
* when culture reaches upper OD, begin dilution until it reaches lower OD value
* repeat